# This workbook will describe the process of predict depth at a Combined Sewer Overflow based on catchment rainfall data. An additional feature (time of day) will be used to test if the model is sensitive enough to predict a diurnal profile based on this real usecase in a small catchment 


In [1]:
#import libaries and set up dependencies
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from __future__ import print_function
import tensorflow as tf

# Uncomment the below lines to check that the correct setup has been made for CUDA acceleration of the neural network

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
#mport plotly.graph_objects as go 

from tensorflow import keras

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.python.keras import backend as k


plt.style.use('ggplot')


# fix random seed for reproducibility
np.random.seed(10)



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2709428404565097155
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22742368256
locality {
  bus_id: 1
  links {
  }
}
incarnation: 693710866253948349
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
]
Default GPU Device: /device:GPU:0
Num GPUs Available:  1


## The first step is to import and format teh recorded historical data so that it can be used for modelling

In [2]:
#Load rainfall dataframe
df_rain = pd.read_csv("/home/nosamaj/Big Disk/Linuxstuff/Minworth Rain/df_rain.csv")
df_rain['date_time'] = pd.to_datetime(df_rain['date_time'],dayfirst=True)
#df_rain.to_hdf("/media/nosamaj/Big Disk/Linuxstuff/Minworth Rain/df_rain.hd5",key='df_rain', mode='w')
df_rain.dtypes
df_rain.head(-5)

FileNotFoundError: [Errno 2] No such file or directory: '/home/nosamaj/Big Disk/Linuxstuff/Minworth Rain/df_rain.csv'

### Based on GIS assessment and catchment knowledgfe me need profiles 9 10 and 13 from the inforworks model. These correspond to RG35, RG40 and RG64 

In [ ]:
df_rain_local = df_rain[['date_time','RG35','RG40','RG64']]
df_rain_local.head()
df_rain_local.dtypes

### Importing and formatting the CSO level data

In [ ]:
data_from = pd.to_datetime('2017-01-01 00:00')
data_to = pd.to_datetime('2021-01-01 00:00')

df_logger = pd.read_csv('./dataset/STWW11075.csv')
df_logger['DateTime'] = pd.to_datetime(df_logger['DateTime'],dayfirst=True, infer_datetime_format=True)

### There are some clear recording issues in this data that need to be cleaned up. As an initial pass we are going to just set any readings <0 to 0 as this is the lowest phsically possible value. This will still represent a real world quality dataset without throwing off the training process with outliers

In [ ]:
df_logger["C1 Depth (mm)"] = df_logger["C1 Depth (mm)"].clip(lower=0)
#fig = px.line(df_logger, x="DateTime", y= "C1 Depth (mm)", width = 1600)
#fig.show()

### Merging the level data with rainfall data based on the timestamp to create a single table for the recorded period. The level data begins incorrectly formatted. The period for the train test procedure will be 01/01/2017 to 01/01/2021



In [ ]:
df_dataset = pd.merge(df_logger, df_rain_local, left_on = "DateTime", right_on = "date_time")
df_dataset = df_dataset.drop("date_time", axis = 1)
df_dataset.head(-5)

In [ ]:
data_from = pd.to_datetime('2017-01-01 00:00')
data_to = pd.to_datetime('2021-01-01 00:00')



df_input = df_dataset[df_dataset['DateTime'] >=data_from]
df_testtrain = df_input[df_input['DateTime'] < data_to]
df_testtrain.head()
df_unseen = df_dataset[df_dataset['DateTime'] >= data_to]



## Data Transformation

### The data for the date and time parameters needs to be transformed into numerical features for the LSTM to function. Creating columns for the month, weekday, hour and minute separately should suffice

In [ ]:
#set the field containing timestamp values as index
df_transform = df_dataset.set_index('DateTime')
#fetch the timestamp componnents and create columns
df_transform['year'] = df_transform.index.year
df_transform['month'] = df_transform.index.month
df_transform['day'] = df_transform.index.weekday
df_transform['week'] = df_transform.index.week
df_transform['hour'] = df_transform.index.hour
df_transform['minute'] = df_transform.index.minute
#see if it worked 
df_transform.head()

In [ ]:
# plot dataset for visualiseation
dataset   = df_transform.values
dataset   = df_transform.astype('float32')
#plt.plot(dataset)

In [ ]:
# normalize the dataset
scaler  = MinMaxScaler()
dataset = scaler.fit_transform(dataset)

The data set comtains a depth in mm and therefore of the order 100, rainfall intesnisty in mm/hr of the order 1 and a decimal time vaule. Therefore it is crucial that the data is standardsised as the significance of the inputs is crucial.

In [ ]:
# split into train and test sets this is a 90% train 10% test 
train_size  = int(len(dataset) * 0.75)
test_size   = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
#count datapoints to check the split
print(len(train), len(test))

In [ ]:
#plot the datasets for visualisation
#print('----------------- TRAINING DATA -----------------')
#plt.plot(train)
#plt.show()
#print('----------------- TEST DATA -----------------')
#plt.plot(test)
#plt.show()


As this is a timeseries sequience problem we need to create a sliding window to determine what period of data we are going to take into account to make predictions. 

In [ ]:
# This function creates a sliding window of the dataset.
def create_dataset(dataset, sliding_window=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-sliding_window-1):
        a = dataset[i:(i+sliding_window), [1,2,3]]
        dataX.append(a)
        dataY.append(dataset[i + sliding_window, 0])
    return np.array(dataX), np.array(dataY)


In [ ]:
# use a n-10 sliding window equivalent to 2 hours of historical data - i.e. we are applying the winddow in this cell
slide_window   = 240
trainX, trainY = create_dataset(train, slide_window)
testX, testY   = create_dataset(test, slide_window)    

## We can now reshape teh datasets to be used as tensors for the keras dequential LSTM model and train it based on the training dataset

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 3, trainX.shape[1]))
testX  = np.reshape(testX, (testX.shape[0], 3, testX.shape[1]))

In [ ]:
#Setup the LSTM
model = tf.keras.Sequential()
model.add(LSTM(3, input_dim=slide_window, return_sequences = True))
model.add(LSTM(3, input_dim=slide_window, return_sequences = True))
model.add(LSTM(3, input_dim=slide_window))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=150, verbose=1)

I haven't had time to play around with hyperparameters. Batch size has been set to 100 in order to make full use of the GPU accelleration

## We can finally see what the predictiosn are doing for us, firstly in the training dataset, and then in the test dataset

In [ ]:
trainPredict = model.predict(trainX)
testPredict  = model.predict(testX)
 

#

In [ ]:
plt.plot(trainY)
plt.plot(trainPredict)

In [ ]:
plt.plot(testY)
plt.plot(testPredict)

In [ ]:



df_test = pd.DataFrame()
df_test['obs'] = testY
df_test['pred'] = testPredict
df_test.head()

In [ ]:

df_train = pd.DataFrame()
df_train['obs'] = trainY
df_train['pred'] = trainPredict
df_testtrain = df_train.append(df_test)
df_testtrain['datetime'] = df_dataset['DateTime']
df_testtrain.sort_values(by='datetime')
df_testtrain.to_csv('results.csv')

In [ ]:
import plotly.express as px
fig = px.line(data_frame= df_testtrain, x='datetime', y = ['obs', 'pred'])
fig.show()